In [1]:
# Set environmental variables with google service credentials
import os
%env GOOGLE_APPLICATION_CREDENTIALS=/home/yexi/interview_questions/ntuc_iowa_liquor/bq_sql_key.json
print(os.getenv('GOOGLE_APPLICATION_CREDENTIALS'))

env: GOOGLE_APPLICATION_CREDENTIALS=/home/yexi/interview_questions/ntuc_iowa_liquor/bq_sql_key.json
/home/yexi/interview_questions/ntuc_iowa_liquor/bq_sql_key.json


In [2]:
# import necessary libraries for analysis
import pandas as pd
import numpy as np
from scipy.stats import norm, t
from statsmodels.stats.weightstats import ztest
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from google.cloud import bigquery

bqclient = bigquery.Client()

In [3]:
# Test BQ Connect
def implicit():
    from google.cloud import storage

    # If you don't specify credentials when constructing the client, the
    # client library will look for credentials in the environment.
    storage_client = storage.Client()

    # Make an authenticated API request
    buckets = list(storage_client.list_buckets())
    print(buckets)

implicit()

[]


In [4]:
# Download query results.
daily_clv_by_country = """
WITH survey_after_event_voucher AS
(
    SELECT
        events.customer_id AS customer_id,
        event_ts,
        country,
        survey_response_ts,
        action_type,
        survey_rating,
        customer_value_index
    FROM `bq-sql-practice.foodpanda_interview.events` AS events
    LEFT JOIN `bq-sql-practice.foodpanda_interview.customer_surveys` AS surveys
        ON events.customer_id = surveys.customer_id
    WHERE survey_response_ts > event_ts
)
SELECT 
    customer_id,
    action_type,
    CAST(DATE_TRUNC(survey_response_ts, day) AS date) AS survey_response_date,
    CAST(DATE_TRUNC(event_ts, day) AS date) AS event_date,
    DATE_DIFF(CAST(DATE_TRUNC(survey_response_ts, day) AS date), CAST(DATE_TRUNC(event_ts, day) AS date), day) AS days_diff,
    country,
    survey_rating,
    customer_value_index
FROM survey_after_event_voucher
"""

daily_clv_by_country = (
    bqclient.query(daily_clv_by_country)
    .result()
    .to_dataframe(
        create_bqstorage_client=True,
    )
)

In [5]:
daily_clv_by_country

,customer_id,action_type,survey_response_date,event_date,days_diff,country,survey_rating,customer_value_index
0,oWiUMwYspToSkubBzcxFZQ==,no_action,2020-08-06,2020-07-30,7,Taiwan,1,1
1,GhQotR2IU7PE6cU/YcTQaw==,no_action,2020-07-01,2020-07-01,0,Taiwan,1,1
2,moHidBgzRZQWEiFbXOdh0w==,no_action,2020-07-01,2020-07-01,0,Taiwan,0,1
3,jjp5L1+45tcSdPwZ7hC2Zg==,no_action,2020-07-01,2020-07-01,0,Taiwan,1,1
4,6IPHozmQJY3qzWiHIMHAmw==,no_action,2020-07-01,2020-07-01,0,Taiwan,1,1
...,...,...,...,...,...,...,...,...
42475,KKobTqC+jvIWpOo8pXHvuw==,absolute_voucher,2020-08-05,2020-07-30,6,Thailand,0,4
42476,RJM6quxs2UdMad4SG97QUw==,absolute_voucher,2020-08-05,2020-07-30,6,Thailand,0,4
42477,0LFvmtI6/R7/ZNyVlM/XVg==,absolute_voucher,2020-08-05,2020-07-29,7,Thailand,1,4
42478,7VmukLx0tgP43RmSl+ABBw==,absolute_voucher,2020-08-05,2020-07-29,7,Bangladesh,0,4


In [6]:
daily_clv_by_country.groupby('customer_value_index')['survey_rating'].mean()

customer_value_index
-1    0.305750
 1    0.483858
 2    0.645442
 3    0.652664
 4    0.653204
Name: survey_rating, dtype: float64

In [8]:
daily_clv_by_country.groupby('country')['survey_rating'].mean()

country
Bangladesh    0.523330
Malaysia      0.551308
Pakistan      0.548553
Taiwan        0.595335
Thailand      0.585235
Name: survey_rating, dtype: float64

In [7]:
pd.pivot_table(
    data=daily_clv_by_country,
    index='country',
    columns='customer_value_index',
    values='survey_rating',
    aggfunc=np.mean
)

customer_value_index,-1,1,2,3,4
country,,,,,
Bangladesh,0.288684,0.481058,0.644485,0.599727,0.627517
Malaysia,0.244755,0.459960,0.636552,0.669139,0.656331
Pakistan,0.319355,0.485988,0.640681,0.661017,0.624041
Taiwan,0.289655,0.501009,0.659483,0.665906,0.655955
Thailand,0.334351,0.491047,0.647750,0.648649,0.660928


In [9]:
pd.pivot_table(
    data=daily_clv_by_country,
    index='country',
    columns='customer_value_index',
    values='customer_id',
    aggfunc=lambda x: len(x.unique())
)

customer_value_index,-1,1,2,3,4
country,,,,,
Bangladesh,422,2684,1006,634,229
Malaysia,286,3008,1435,1323,737
Pakistan,612,4199,1859,1440,663
Taiwan,145,1973,1153,1300,1022
Thailand,653,5056,2893,3222,2925
